In [37]:
import requests


#% AUTHORISATION SECTION

CLIENT_ID = 'd4f47f0943cc48c584edb906357648a2'
CLIENT_SECRET = '797dc982d7924038a228159b23a8b6b8'
AUTH_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'


# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# create header to input to requests
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}



### now want to only get the playlists which are the monthly ones

Since I've been organised, they will all begin with 201 or 202! This could be a tricky one otherwise, a way to make it more flexible might be to create a dictionary with the possible month abbreviations or similar and grab all those.


In [38]:
#% TAKE USERNAME
user = input('Type your username: ')

# INCOMPLETE!
# filter playlists containing terms: give a comma separated list
filter_input = input('Playlist filter: ').split(',')
playlist_filter = {x.strip() for x in filter_input}

Type your username: geoffliddell
Playlist filter: 201, 202


In [41]:
#% GET ALL USER PLAYLIST URIs

# create a set which will contain strings common to all target playlists, i.e. those organised by month
playlist_uris = []
playlist_names = []

# irritating counters to work around the request limit of 50 items
full = 0
offset = 0
lim = 50

while full == 0:
    got_playlists = requests.get(BASE_URL + 'users/' + user + '/playlists', 
                                 headers = headers,
                                 params = {'limit': lim, 'offset': offset}).json()


    for playlist in got_playlists['items']:   
        for filt in playlist_filter:           
            if filt in playlist['name'][0:3]:                          # [0:3] at the end of this line is a special case for MY playlists: removing this will -> less specific search               
                playlist_names.append(playlist['name'])
                playlist_uris.append(playlist['uri'].split(sep = ':')[2]) 


    
    if len(got_playlists['items']) < lim:
        break
        
    offset = offset + lim


playlist_uri_name_dict = dict(zip(playlist_uris, playlist_names))
playlist_uri_name_dict

{'5z2jvoVWwkTOpDJ7qNAi5n': '2021 November',
 '5uIgd8OG3GTsq9m638kpG8': '2021 October',
 '2JSjUF9yquPN6GdmOT1nyu': '2021 September',
 '7DenyceRFQ9hJrvASYFjqw': '2021 August',
 '20rzA08T2MbyUtN8pn83P6': '2021 July',
 '0HtW79mkynItv6yNR4qyhO': '2021 June',
 '3QWRFSzVf5X2F2DlyUOCC7': '2021 May',
 '51vXTziJwZKL9J2NbDzjWZ': '2021 April',
 '2JUR0svkBPuaCqwAndlRrR': '2021 March',
 '70yG5rmgFXLV77UKWdflsU': '2021 Feb',
 '27rAvTxf6yHzxXMxTz7Tth': '2021 Jan',
 '1jbAHcoPfDxbk5WlZFue4x': '2020 December',
 '2HJWZo97Fy0cdh0sv3R0mw': '2020 November',
 '6U4aqtRfi1IDBeDBEkAHYb': '2020 October',
 '1ewVcvyX89eaU01RMCL5M6': '2020 September',
 '6OYVNUd9nzSWqBYkBVU1uV': '2020 August',
 '537bkpNGq7GD6qrnCvP6Bw': '2020 July',
 '3BOnFUKWKlvTd3HGsbIsk6': '2020 June',
 '1FW8mQKFmwDTi6xCa1z1Ul': '2020 May',
 '4ubwQ7CqmOy1z0Pjt0kY3H': '2020 April',
 '2Ys0x9FxFxK95k7leeAWLu': '2020 March',
 '14eI8qN5zUUZPYXUcOHTWi': '2020 Feb',
 '43PfFOOCuMfwXKK6ksmKGR': '2020 Jan',
 '5GbvQQkeC3KphSM4G4F8nD': '2019 December',
 '7AeV

### next thing is probably to collate the track_ids from all those playlists???

could create a dictionary with songs as keys and their respective playlist membership as values. How would that end up dealing with duplicates?

ANS: simply have values as lists which contain all relevant playlists

In [11]:
playlist_uris

['5z2jvoVWwkTOpDJ7qNAi5n',
 '5uIgd8OG3GTsq9m638kpG8',
 '2JSjUF9yquPN6GdmOT1nyu',
 '7DenyceRFQ9hJrvASYFjqw',
 '20rzA08T2MbyUtN8pn83P6',
 '0HtW79mkynItv6yNR4qyhO',
 '3QWRFSzVf5X2F2DlyUOCC7',
 '51vXTziJwZKL9J2NbDzjWZ',
 '2JUR0svkBPuaCqwAndlRrR',
 '70yG5rmgFXLV77UKWdflsU',
 '27rAvTxf6yHzxXMxTz7Tth',
 '1jbAHcoPfDxbk5WlZFue4x',
 '2HJWZo97Fy0cdh0sv3R0mw',
 '6U4aqtRfi1IDBeDBEkAHYb',
 '1ewVcvyX89eaU01RMCL5M6',
 '6OYVNUd9nzSWqBYkBVU1uV',
 '537bkpNGq7GD6qrnCvP6Bw',
 '3BOnFUKWKlvTd3HGsbIsk6',
 '1FW8mQKFmwDTi6xCa1z1Ul',
 '4ubwQ7CqmOy1z0Pjt0kY3H',
 '2Ys0x9FxFxK95k7leeAWLu',
 '14eI8qN5zUUZPYXUcOHTWi',
 '43PfFOOCuMfwXKK6ksmKGR',
 '5GbvQQkeC3KphSM4G4F8nD',
 '7AeVyFhp0jGPW6y5tG0eCU',
 '2q7Gxv5McJO7DVf2mHPNDL',
 '2CNmRMhYT4mIHiZvFRp9xz',
 '3Q1Iqo9fthoDgVQEg0MzCJ',
 '0nAuS1q9odbBsXApRIUbI4',
 '7d4uFKHptZ2povDRtUVbDp',
 '4ycd7r1zROLNhASk12j6Q2',
 '5qe5L9JchHY9VUb6Sy28kf',
 '2fyIH5sJJ9Lttx1naGq6OX',
 '40tw4wxjyTBdu2Ib4ElZRM',
 '7wlkHasVoriXnJ0d7TEIm9',
 '2tpsDyLVYIznj1EYPLjtfP',
 '2a4TnNEbqwKHMQpuaCggso',
 

In [12]:
tracks_playlist_dict = dict()


for playlist_uri in playlist_uris:
    
    # can get away with no iteration here as I believe all my playlists are under 50 tracks long!
    tracks_tmp = requests.get(BASE_URL + 'playlists/' + playlist_uri + '/tracks', 
                          headers = headers,
                          params = {'limit': 50, 'offset': 0}).json()
    
    
    # print a warning if there are more than 50
    if len(tracks_tmp['items']) >= 50:
        print('Warning: Lots of songs in this playlist!', playlist_uri)

    # extract track URIs + add playlist membership to a dict: add as an actual name for now for transparency
    for track in tracks_tmp['items']:
        tracks_playlist_dict.setdefault(track['track']['name'].lower(),
                                        []).append(playlist_uri_name_dict.get(playlist_uri).lower())
        
tracks_playlist_dict

{'sad girlz luv money remix (feat. kali uchis and moliy)': ['2021 november'],
 'watch out': ['2021 november'],
 'only you - frankie francis disco jam edit': ['2021 november'],
 "i'll bet you": ['2021 november'],
 'typical girls': ['2021 november'],
 'spread': ['2021 november'],
 'just another day': ['2021 november'],
 "can't you see (you doin' me wrong)": ['2021 november'],
 'our generation': ['2021 november'],
 'release yourself': ['2021 november'],
 'lemonade': ['2021 november'],
 'fergalicious': ['2021 november'],
 'barry white (ft. zelooperz)': ['2021 november'],
 'turiya & ramakrishna': ['2021 november'],
 'sparrow': ['2021 november'],
 'preservation': ['2021 november'],
 'she had to go': ['2021 november'],
 'burnin ya boa': ['2021 november'],
 'drip master 3000': ['2021 november'],
 'orange groves': ['2021 november'],
 'twisted!': ['2021 november'],
 'charmander': ['2021 november'],
 "don't walk away": ['2021 november'],
 'yoshi orange': ['2021 november'],
 'metal mono': ['2021 n


### HOORAY!!

Now have a dictionary with a track name and the corresponding playlist membership. Unfortunately contained in a list to account for the case (unlikely in my library structure) that a song is in multiple playlist.

Could now also go ahead and re-cluster all those songs into new playlists, but will leave this until after. For now concentrate on creating an easily searchable dictionary to apply the search to!


In [23]:
# concentrate on track name and artist for now...complicated enough as it is! Eventually this 

search_term = input('Type an artist or song here: ').lower()


Type an artist or song here: wat


In [32]:
playlists = ''

# quick search in case where there's an exact track name match
if search_term in tracks_playlist_dict:
    
    # once again sat here accounting for multiple playlists that 100% probably aren't there...
    for playlist in tracks_playlist_dict[search_term]:
        playlists += playlist
        
    print('"' + search_term + '"' + ' is in the playlist ' + tracks_playlist_dict[search_term][0])

# if there's no exact match go back and check for partial matches (slower?)
else:
    
    for track in tracks_playlist_dict:
        if search_term in track:
            print('By "' + search_term + '"' + ' do you mean "' + track + '"?')
    
    #print('"' + search_term + '"' + ' is not in any of your playlist')
#elif search_term not in tracks_playlist_dict:
        
        

By "wat" do you mean "watch out"?
By "wat" do you mean "let's wade in the water"?
By "wat" do you mean "breathing underwater"?
By "wat" do you mean "water no get enemy"?
